<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Без-предварительной-обработки" data-toc-modified-id="Без-предварительной-обработки-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Без предварительной обработки</a></span></li><li><span><a href="#С-предварительной-обработкой" data-toc-modified-id="С-предварительной-обработкой-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>С предварительной обработкой</a></span></li></ul></div>

In [2]:
pip install pymorphy2

     -------------------------------------- 55.5/55.5 kB 580.0 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 8.2/8.2 MB 7.2 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13775 sha256=4ab49e32281ae949a253e65fc291bdbb69df9369b3a9a9dcdace3bcd482a14b8
  Stored in directory: c:\users\vasiliy\appdata\local\pip\cache\wheels\7c\d7\8d\2156234738063e3d4a39ba77dc677046100e62766b53807189
Successfully built docopt
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import re
import pymorphy2
import joblib as pickle

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn import tree

In [17]:
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
%%time
df = pd.read_csv('g:\Temp\lenta-ru-news.csv')

<timed exec>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: total: 43.6 s
Wall time: 52.5 s


In [4]:
df.shape

(800975, 6)

In [5]:
df.tags.value_counts()

Все               453762
Политика           40716
Общество           35202
Украина            22523
Происшествия       19825
                   ...  
Мировой опыт           6
Нацпроекты             6
Вооружение             3
Инновации              1
69-я параллель         1
Name: tags, Length: 94, dtype: int64

In [7]:
df = df[df.topic.isin(['Россия','Мир','Экономика','Спорт','Культура'])].copy()

In [8]:
df.shape

(494804, 6)

In [9]:
df.isna().sum()

url          0
title        0
text         5
topic        0
tags     15328
date         0
dtype: int64

In [10]:
df.topic.value_counts()

Россия       160445
Мир          136621
Экономика     79528
Спорт         64413
Культура      53797
Name: topic, dtype: int64

In [10]:
df.topic.value_counts(normalize=True)

Россия       0.324260
Мир          0.276111
Экономика    0.160726
Спорт        0.130179
Культура     0.108724
Name: topic, dtype: float64

In [11]:
df[df.text.isna()]['title'].values

array(['Прокуратура не\xa0нашла национальной почвы в\xa0нападении на\xa0индийского студента',
       'При штурме дома в\xa0Кизилюрте ранены шестеро',
       'В Стамбуле горит багажное отделение аэропорта "Ататюрк"',
       'РФ вышла из\xa0российско-украинского проекта Ан-70',
       'Гребенщиков заработал 10\xa0тысяч рублей в\xa0омском подземном переходе'],
      dtype=object)

In [12]:
df.dropna(subset=['text'],inplace = True)

In [13]:
df[df.text.isna()]['title'].values

array([], dtype=object)

## Без предварительной обработки

In [24]:
text_transformer = TfidfVectorizer()

In [25]:
text = text_transformer.fit_transform(df['text'])

In [26]:
df['text']

5         Как стало известно агентству Ассошиэйтед Пресс...
6         В зале игровых автоматов в третьем ярусе подзе...
7         Япония приняла решение разморозить кредиты Рос...
8         Британцы отмечают сегодня скорбную дату - втор...
9         В понедельник директор департамента внешних св...
                                ...                        
739168    Президент России Владимир Путин, выступая на з...
739169    Президент России Владимир Путин организовал дл...
739171    Протесты движения «желтых жилетов» в Париже сн...
739174    Швейцарский горнолыжник Марк Гизин неудачно пр...
739175    Президент России Владимир Путин рассказал, что...
Name: text, Length: 494799, dtype: object

In [35]:
text.shape

(494799, 821732)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.20, random_state=42)

In [37]:
clf = tree.DecisionTreeClassifier()

CPU times: total: 0 ns
Wall time: 0 ns


In [38]:
%%time
clf.fit(X_train,y_train)

CPU times: total: 2h 41min 43s
Wall time: 8h 43min 53s


DecisionTreeClassifier()

In [39]:
pred = clf.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test,pred,average='weighted'))

Качество модели по метрике F1 0.7910159246454394


## С предварительной обработкой

In [40]:
# меняем токинизатор
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vasiliy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
nltk.download('stopwords')
stop_russian = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vasiliy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
stop_russian

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [18]:
# функция удаления пунктуации и цифр
def remove_trash(list): 
    pattern = r'[^А-Яа-я]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
#       print (list)
    except Exception as e:
      print(e)
    return list

In [19]:
# очищаем текст от мусора
df['text_clean'] = remove_trash(df['text'])

In [20]:
df.isna().sum()

url               0
title             0
text              0
topic             0
tags          15328
date              0
text_clean        0
dtype: int64

In [26]:
# проводим лематизацию
morph = pymorphy2.MorphAnalyzer()

In [27]:
# функция лемматизации
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=1:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [28]:
%%time
df['text_clean_normal'] = df.apply(lemmatize,axis=1)

CPU times: total: 7h 59min 34s
Wall time: 8h 6min 34s


In [29]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000)
text_norm = text_transformer.fit_transform(df['text_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, df['topic'], test_size=0.20, random_state=42)

In [30]:
%%time
clf2 = GradientBoostingClassifier()
clf2.fit(X_train,y_train)

CPU times: total: 4h 23min 56s
Wall time: 4h 27min 9s


GradientBoostingClassifier()

In [31]:
pred_norm = clf2.predict(X_test)

In [32]:
print('Качество модели по метрике F1 после токинезации, очистки и лемматизации', f1_score(y_test,pred_norm,average='weighted'))

Качество модели по метрике F1 после токинезации, очистки и лемматизации 0.872969297030309


In [34]:
# сохраняем модель в файл
with open('data/GB_clf2.pkl', 'wb') as f:
    pickle.dump(clf2, f)

with open('data/transformer.pkl', 'wb') as f:
    pickle.dump(text_transformer, f)